In [ ]:
%load_ext autoreload
%autoreload 2

In [25]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from instruments.utils import *
from instruments.FSW_0010 import *
from instruments.PXIe_5170R import *
import time

In [30]:
file_name = '../data/scan_1GHz'
file = file_name + '.h5'
ref = 5.87045 
step = 0.0002
window     = 100           #length of half of the interval on x axis
step       = 0.0002        #length of a single step during the frequency sweep (GHz)

In [ ]:
trigger = dict(
    trigger_type   = 'IMMEDIATE', #'EDGE', 'IMMEDIATE' or 'DIGITAL'
    trigger_source = '0',
    trigger_slope  = 'POSITIVE', #'POSITIVE' or 'NEGATIVE'
    trigger_level  = '0.0',
    trigger_delay  = '0.0'
)

daq =  PXIeSignalAcq("PXI1Slot2", trigger=trigger, records=1, channels=[0,1], sample_rate=5e7, length=1000)
with FSWSynt("COM12") as synt:
    print(synt.get_ID())
    
    for i in range(-window, window):
        freq = ref + i*step
        print(synt.set_freq(freq))
        time.sleep(0.005)
        #print(synt.get_freq(freq))
        print(i)
        daq.acq()   #DA TESTARE
    
    daq.storage_hdf5(file_name + '.h5')

daq.close()

In [32]:
I, Q = get_hdf5(file)
print('%d frequencies considered, for each one %d points have been taken.' %(len(I), len(I[0])))

10 frequencies considered, for each one 1000 points have been taken.


In [ ]:
I_mean, Q_mean = get_mean(file)
big_plot_from_array(I_mean, Q_mean, ref, step)

In [ ]:
x = []
for i in range(-int(len(Q_mean)/2), int(len(Q_mean)/2)): 
    x.append(ref + i*step) 
#decide begin and end by looking at the plot
index_max = der_IQ(x, I_mean, Q_mean, begin = 50, end = 150, plot = True)